In [1]:
from veering import taut_polytope
from veering import taut
from veering import taut_homology
from veering import taut_carried
from veering import fundamental_domain
from veering import taut_polynomial
from veering import transverse_taut
from veering import carried_surface

from sage.plot.contour_plot import ContourPlot

from itertools import combinations
from itertools import permutations

import numpy as np

import time

m = mathematica

# Notes:
#  - Maybe add a get_hom_vars function

In [247]:
# Homology dimension 1 example:
#sig = 'cPcbbbiht_12'
# Homology dimension 2 example: (Hironaka example)
#sig = 'eLMkbcddddedde_2100'
# Homology dimension 3 example:
#sig = 'ivLLQQccdgffhghhvvaaaaavv_01111220'
# Homology dimension 4 example:
sig = 'ivvPQQcfhghgfghfaaaaaaaaa_01122000'

In [253]:
get_Thurston_norm(sig)

x0 + x1 + x2 + x3

In [243]:
def get_extreme_surfs(sig): # NOT SURE IF I ACTUALLY NEED THIS FUNCTION FOR ANYTHING...
    '''This function gets the branch equation 
    (non-negative 2-chain) form of the surfaces 
    which span the fibered cone.'''
    tri, angle = taut.isosig_to_tri_angle(sig)
    extreme_rays = taut_polytope.cone_in_homology(sig) # rays (in homology) spanning the fibered cone.
    rays = taut_polytope.taut_rays(sig) # These are rays spanning the cone of carried surfaces (not sure exactly what these are tbh...)
    P = taut_polytope.projection_to_homology(sig) # This is the matrix to multiply by to get branched surface tuple to a second homology class.
    extreme_surfs = []
    for ray in rays: # this for loop stores the surfaces (in terms of their branch equations) that define the boundary of the cone
        if P*ray in extreme_rays:
            temp = 0
            for surf in extreme_surfs:
                if P*surf != P*ray:
                    temp = temp + 1
            if temp == len(extreme_surfs):
                extreme_surfs.append(ray)
            
    if tri.homology().rank() > 1 and [tuple(extreme_rays[0]), tuple(extreme_rays[1])] != [tuple(P*extreme_surfs[0]), tuple(P*extreme_surfs[1])]:
        extreme_surfs.reverse()
            
    return extreme_surfs

def get_fibered_cone(sig):
    '''This function gets the fibered cone
    in homology of the given veering triangulation.'''
    extreme_rays = taut_polytope.cone_in_homology(sig)
    return Cone(extreme_rays)

def get_fibered_cone_plot(sig):
    '''This function simply plots the fibered
    cone. It will only work if the homology is
    rank 2 or 3.'''
    tri, angle = taut.isosig_to_tri_angle(sig)
    rank = tri.homology(1).rank()
    if rank == 2 or rank == 3:
        return get_fibered_cone(sig).plot(mode='box', xmin = -10, xmax = 10, ymin = -10, ymax = 10, zmin = -10, zmax = 10, ray_thickness=2, point_size=10)
    
    return 'This function only works for manifolds with H_2(M,dM) (equivalently H_1(M)) dimension 2 or 3.'

def get_spec(sig):
    '''Auxiliary function for the "get_spec_at_e" function.
    Returns the specialization of the taut polynomial at a
    point in second homology. Also used in the get_dila_mathematica
    function.'''
    
    tri, angle = taut.isosig_to_tri_angle(sig)
    rank = tri.homology(1).rank()
    hom_vars = var(','.join('x%s'%i for i in range(rank))) # variables representing an arbitrary point in homology
    if type(hom_vars) != tuple: # Check if singleton or not (just we don't get an iterable error in dimension 1)
        hom_vars = (hom_vars,)
    poly = taut_polynomial.taut_polynomial(tri, angle)
    unspec_monos = poly.monomials()
    monos = [] # the monomials in the specialization
    for i in range(poly.number_of_terms()):
        monos.append(unspec_monos[i].lc()*x^(sum([poly.exponents()[i][j]*hom_vars[j] for j in range(len(hom_vars))])))
        
    spec = 0
    for i in range(len(monos)):
        spec = spec + monos[i]*poly.monomial_coefficient(poly.monomials()[i])
        
    return spec

def get_spec_at_e(sig):
    '''Returns the specialization at e of the taut
    polynomial. Used in the "get_fibered_cone_and_levset_plot" 
    function as well as the "get_minimal_direction" function.'''
    return get_spec(sig)(x = e)

def get_fibered_cone_and_levset_plot(sig):
    '''This function plots both the fibered
    cone and the level set of the normalized
    dilatation where it equals the Thurston norm.
    Only works if the homology is dimension 2 or 3.'''
    tri, angle = taut.isosig_to_tri_angle(sig)
    rank = tri.homology(1).rank()
    if rank == 3:
        levset = implicit_plot3d(get_spec_at_e(sig) == 0, (-10,10), (-10,10), (-10,10), cmap=['green'])
        Plot = levset + get_fibered_cone_plot(sig)
        return Plot
    
    if rank == 2:
        levset = implicit_plot(get_spec_at_e(sig) == 0, (-10,10), (-10,10), cmap=['green'])
        Plot = levset + get_fibered_cone_plot(sig)
        return Plot
    
    return 'This function only works for manifolds with H_2(M,dM) (equivalently H_1(M)) dimension 2 or 3.'

In [250]:
def get_2_chain_from_point_in_homology(sig):
    '''This function produces a 2-chain which is the
    preimage of an arbitrary homology class, making
    sure that the obtained 2-chain is not only a 
    preimage under the projection from 2-chains to
    homology, but also that it represents a homology
    class; i.e. it is in the kernel of the boundary map'''
    tri, angle = taut.isosig_to_tri_angle(sig)
    rank = tri.homology(1).rank()
    hom_vars = var(','.join('x%s'%i for i in range(rank))) # variables representing an arbitrary point in homology
    if type(hom_vars) != tuple: # Check if singleton or not (just we don't get an iterable error in dimension 1)
        hom_vars = (hom_vars,)
    #print(hom_vars) # For testing
    
    P = taut_polytope.projection_to_homology(sig)
    #print(P) # For testing
    num_triangles = tri.countTriangles() # This is the number of variables we need to store for our 2-chain.
    
    C2M_to_C1M = get_C2M_to_C1M(sig)
    chain_vars = var(','.join('a%s'%i for i in range(num_triangles))) # The 2-chain variables
    preimage_in_kernel = vector(chain_vars)
    #print(preimage_in_kernel) # For testing
    
    eqns = []
    for term in list(C2M_to_C1M*preimage_in_kernel):
        eqns.append(term == 0) # we want our 2-chain to be in the kernel of the boundary map so that it actually represents a homology class
    #print(eqns) # For testing
    for i in range(rank):
        eqns.append(list(P*preimage_in_kernel)[i] == hom_vars[i]) # and we want its image in homology to be our homology class
    #print(eqns) # For testing
    solution = solve(eqns, chain_vars)
    
    preimage = vector([solution[0][i].rhs() for i in range(len(solution[0]))])
    
    return preimage

In [202]:
def get_Thurston_norm(sig):
    '''This function gets the formula for
    the Thurston norm of our 3-manifold for
    an arbitrary homology class'''
    extreme_rays = taut_polytope.cone_in_homology(sig) # These are rays spanning the cone of carried classes 
        
    surf = get_2_chain_from_point_in_homology(sig)
    prenorm = 0
    for tri in surf:
        prenorm = prenorm + tri
        
    return prenorm/2

In [7]:
def get_minimal_direction(sig):
    
    return None

In [8]:
def get_min_norm_dila_log_mathematica_approx(sig):
    
    return None

In [9]:
def get_C2M_to_C1dM(sig):
    
    return None

In [125]:
def get_C2M_to_C1M(sig):
    '''This function returns the standard boundary map
    on 2-chains, with orientations agreeing with the
    veering coorientations.'''
    tri, angle = taut.isosig_to_tri_angle(sig)
    tet_vert_coorientations = taut_carried.is_transverse_taut(tri, angle, return_type = "face_coorientations") # This gives a list telling us whether the RHR orientations of the regina numbering agrees or disagrees with the veering coorientation. (See Veering repo for details)
    
    reg_mat = tri.boundaryMap(2) # This is the Regina generated boundary map, but the signs are according to traversing the triangles with the RHR wrt the Regina vertex ordering.

    nrows = reg_mat.rows()
    ncols = reg_mat.columns()
    
    # We first make a matrix
    mat = [[0 for i in range(ncols)] for j in range(nrows)]
    for i in range(nrows):
        for j in range(ncols):
            mat[i][j] = int(str(reg_mat.entry(i,j)))
        
    # And we change the signs to agree with the veering coorientation (multiply entire columns by the discrepancy factors +-1 recorded in tet_vert_coorientations)
    for i in range(ncols):
        for j in range(nrows):
            mat[j][i] = tet_vert_coorientations[i]*mat[j][i]
    
    return matrix(mat)